In [ ]:
# 1. Gerekli kütüphaneleri kuruyoruz
!pip install -q -U google-generativeai langchain-google-genai langchain faiss-cpu langchain_community

# 2. Veri setimizi (tarifler.txt) içeren GitHub reponuzu klonluyoruz
!git clone https://github.com/mustafadevrim/akbank-genai-bootcamp-proje

# 3. Dosyaların gelip gelmediğini kontrol ediyoruz
!ls akbank-genai-bootcamp-proje

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
Cloning into 'akbank-genai-bootcamp-proje'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 11.68 KiB | 351.00 KiB/s, done.
Resolving deltas: 100

In [ ]:
import os
from google.colab import userdata
import google.generativeai as genai

# Proje Mimarisi için LangChain bileşenlerini import et
from langchain_google_genai import GoogleGenerativeAIEmbeddings  # Embedding model
from langchain_google_genai import ChatGoogleGenerativeAI       # Generation model
from langchain.document_loaders import TextLoader                 # Veri yükleyici
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Veri parçalayıcı
from langchain.vectorstores import FAISS                          # Vektör veritabanı
from langchain.chains import RetrievalQA                          # RAG zinciri

# --- API ANAHTARI YÜKLEME ---
try:
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    print("✅ API Anahtarı ve Kütüphaneler Başarıyla Yüklendi!")
    print("RAG Mimarisi için hazırız.")
except Exception as e:
    print(f"❌ HATA: API Anahtarı yüklenemedi. 'Secrets' (🔑) bölümünü kontrol edin. Hata: {e}")

✅ API Anahtarı ve Kütüphaneler Başarıyla Yüklendi!
RAG Mimarisi için hazırız.


In [ ]:
# --- 1. Veri Yükleme (Load) ---
# Klonladığımız reponun içindeki tarifler.txt dosyasını yüklüyoruz.
loader = TextLoader("akbank-genai-bootcamp-proje/tarifler.txt")
documents = loader.load()

# --- 2. Veri Parçalama (Split) ---
# Tarifleri birbirinden ayırmak ve RAG için optimize etmek amacıyla
# metni '---' (tarif ayıracı) ile bölüyoruz.
text_splitter = RecursiveCharacterTextSplitter(
    separators=["---", "\n\n", "\n", " "], # Tarifleri '---' ile ayır
    chunk_size=1000, # Her bir parçanın maksimum boyutu
    chunk_overlap=200 # Parçalar arası bindirme, anlamsal bütünlük için
)
texts = text_splitter.split_documents(documents)

# Kaç adet metin parçası (chunk) oluştuğunu görelim
print(f"{len(texts)} adet metin parçası (chunk) oluşturuldu.")

# --- 3. Embedding & Vektör Veritabanı Oluşturma (Embed & Store) ---
# Proje dosyasında önerilen Google embedding modelini kullanıyoruz
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Parçalanmış metinleri (texts) ve embedding modelini kullanarak
# Proje dosyasında önerilen FAISS vektör veritabanını oluşturuyoruz.
try:
    vector_store = FAISS.from_documents(texts, embeddings)
    print("\n✅ Vektör veritabanı (FAISS) başarıyla oluşturuldu!")
    print("Artık chatbot'a soru sormaya hazırız.")
except Exception as e:
    print(f"\n❌ HATA: Vektör veritabanı oluşturulamadı. Hata: {e}")
    print("API Anahtarınızın 'Generative AI API' için yetkisi olduğundan emin olun.")

24 adet metin parçası (chunk) oluşturuldu.

❌ HATA: Vektör veritabanı oluşturulamadı. Hata: Error embedding content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0 [violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerDayPerProjectPerModel-FreeTier"
}
violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "Em

In [ ]:
# Açık kaynak embedding modellerini kullanmak için
!pip install -q -U sentence-transformers

In [ ]:
# Google yerine Hugging Face (açık kaynak) embedding modelini import ediyoruz
from langchain_community.embeddings import HuggingFaceEmbeddings

# --- 1. Veri Yükleme (Load) ---
# (Bu kısım aynı, zaten çalışmıştı)
loader = TextLoader("akbank-genai-bootcamp-proje/tarifler.txt")
documents = loader.load()

# --- 2. Veri Parçalama (Split) ---
# (Bu kısım da aynı, zaten çalışmıştı)
text_splitter = RecursiveCharacterTextSplitter(
    separators=["---", "\n\n", "\n", " "],
    chunk_size=1000,
    chunk_overlap=200
)
texts = text_splitter.split_documents(documents)

# Kaç adet metin parçası (chunk) oluştuğunu görelim
print(f"{len(texts)} adet metin parçası (chunk) oluşturuldu.") # 24 adet görmeliyiz

# --- 3. Embedding & Vektör Veritabanı Oluşturma (REVİZE EDİLDİ) ---

# Proje dosyasında önerilen açık kaynak model seçeneğini kullanıyoruz
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

print("\nAçık kaynak embedding modeli (Hugging Face) yüklendi.")

# Parçalanmış metinleri ve YENİ embedding modelini kullanarak
# FAISS vektör veritabanını oluşturuyoruz.
try:
    vector_store = FAISS.from_documents(texts, embeddings)
    print("\n✅ Vektör veritabanı (FAISS) başarıyla oluşturuldu!")
    print("Artık chatbot'a soru sormaya hazırız.")
except Exception as e:
    print(f"\n❌ HATA: Vektör veritabanı oluşturulamadı. Hata: {e}")

24 adet metin parçası (chunk) oluşturuldu.


/tmp/ipython-input-1148925864.py:26: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


Açık kaynak embedding modeli (Hugging Face) yüklendi.

✅ Vektör veritabanı (FAISS) başarıyla oluşturuldu!
Artık chatbot'a soru sormaya hazırız.


In [ ]:
import google.generativeai as genai

print("API anahtarınızın (generateContent) erişebildiği modeller listeleniyor:")
print("---------------------------------------------------------------")
model_listesi = []
try:
    for model in genai.list_models():
        # Biz 'generateContent' (sohbet) yapabilen modelleri arıyoruz
        if 'generateContent' in model.supported_generation_methods:
            model_listesi.append(model.name)
            print(f"✅ Bulundu: {model.name}")

    if not model_listesi:
        print("\n❌ HATA: 'generateContent' metodunu destekleyen model bulunamadı.")
        print("Lütfen API anahtarınızın Google AI Studio'da doğru şekilde etkinleştirildiğinden emin olun.")
    else:
        print(f"\n👉 Öneri: Bir sonraki adımda bu listedeki modellerden birini kullanın.")

except Exception as e:
    print(f"\n❌ HATA: Modeller listelenemedi. Hata: {e}")
    print("API anahtarınızda veya bağlantınızda bir sorun olabilir.")

API anahtarınızın (generateContent) erişebildiği modeller listeleniyor:
---------------------------------------------------------------
✅ Bulundu: models/gemini-2.5-pro-preview-03-25
✅ Bulundu: models/gemini-2.5-flash-preview-05-20
✅ Bulundu: models/gemini-2.5-flash
✅ Bulundu: models/gemini-2.5-flash-lite-preview-06-17
✅ Bulundu: models/gemini-2.5-pro-preview-05-06
✅ Bulundu: models/gemini-2.5-pro-preview-06-05
✅ Bulundu: models/gemini-2.5-pro
✅ Bulundu: models/gemini-2.0-flash-exp
✅ Bulundu: models/gemini-2.0-flash
✅ Bulundu: models/gemini-2.0-flash-001
✅ Bulundu: models/gemini-2.0-flash-exp-image-generation
✅ Bulundu: models/gemini-2.0-flash-lite-001
✅ Bulundu: models/gemini-2.0-flash-lite
✅ Bulundu: models/gemini-2.0-flash-preview-image-generation
✅ Bulundu: models/gemini-2.0-flash-lite-preview-02-05
✅ Bulundu: models/gemini-2.0-flash-lite-preview
✅ Bulundu: models/gemini-2.0-pro-exp
✅ Bulundu: models/gemini-2.0-pro-exp-02-05
✅ Bulundu: models/gemini-exp-1206
✅ Bulundu: models/gemin

In [ ]:
# --- 1. Generation Model'i Yükleme  ---
# API anahtarınızın desteklediği 'gemini-flash-latest' modelini kullanıyoruz.
llm = ChatGoogleGenerativeAI(model="models/gemini-flash-latest",
                             temperature=0.7,
                             top_p=0.85)

# --- 2. RAG Zincirini (Pipeline) Oluşturma ---
rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

print("✅ RAG Pipeline (Gemini Flash Latest + FAISS + LangChain) başarıyla kuruldu.")

# --- 3. RAG Pipeline'ını Test Etme ---
print("\n--- TEST BAŞLIYOR ---")
soru = "Karnıyarık nasıl yapılır? Malzemeleri nelerdir?"
print(f"Soru: {soru}")

# RAG zincirini çalıştır
try:
    cevap = rag_pipeline.invoke(soru)
    print("\nCevap:")
    print(cevap['result'])
except Exception as e:
    print(f"\n❌ HATA: Model sorgulanamadı. Hata: {e}")

✅ RAG Pipeline (Gemini Flash Latest + FAISS + LangChain) başarıyla kuruldu.

--- TEST BAŞLIYOR ---
Soru: Karnıyarık nasıl yapılır? Malzemeleri nelerdir?

Cevap:
Bu bağlamda Karnıyarık tarifi bulunmamaktadır. Bu yüzden sorunuzun cevabını bilmiyorum.


In [ ]:
# --- 1. Gerekli Import İşlemleri ---
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

print("Daha güçlü bir embedding modeli (multilingual) yükleniyor...")
print("Bu işlem 1-2 dakika sürebilir, model indirilecek...")

# --- 2. GÜÇLENDİRİLMİŞ EMBEDDING MODELİ ---
# Türkçe'yi daha iyi anlayan, popüler bir açık kaynak model kullanıyoruz.
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

print(f"\n✅ {model_name} modeli başarıyla yüklendi.")

# --- 3. VEKTÖR VERİTABANINI YENİDEN OLUŞTURMA ---

try:
    vector_store = FAISS.from_documents(texts, embeddings)
    print("✅ Vektör veritabanı (FAISS) YENİ model ile başarıyla oluşturuldu!")
except Exception as e:
    print(f"\n❌ HATA: Vektör veritabanı oluşturulamadı. Hata: {e}")

# --- 4. RAG PİPELİNE'INI YENİDEN KURMA ---

llm = ChatGoogleGenerativeAI(model="models/gemini-flash-latest",
                             temperature=0.7,
                             top_p=0.85)

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

print("✅ RAG Pipeline (Gemini + Yeni Embedding) başarıyla kuruldu.")

# --- 5. RAG PİPELİNE'INI TEKRAR TEST ETME ---
print("\n--- TEST (YENİ MODEL İLE) BAŞLIYOR ---")
soru = "Karnıyarık nasıl yapılır? Malzemeleri nelerdir?"
print(f"Soru: {soru}")

try:
    cevap = rag_pipeline.invoke(soru)
    print("\nCevap:")
    print(cevap['result'])
except Exception as e:
    print(f"\n❌ HATA: Model sorgulanamadı. Hata: {e}")

Daha güçlü bir embedding modeli (multilingual) yükleniyor...
Bu işlem 1-2 dakika sürebilir, model indirilecek...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


✅ sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 modeli başarıyla yüklendi.
✅ Vektör veritabanı (FAISS) YENİ model ile başarıyla oluşturuldu!
✅ RAG Pipeline (Gemini + Yeni Embedding) başarıyla kuruldu.

--- TEST (YENİ MODEL İLE) BAŞLIYOR ---
Soru: Karnıyarık nasıl yapılır? Malzemeleri nelerdir?

Cevap:
Elimdeki mevcut metinlerde Karnıyarık tarifi ve malzemeleri bulunmamaktadır.


In [ ]:
# --- 1. Gerekli Import İşlemleri ---
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document # Belgeleri manuel oluşturmak için

print("RAG Pipeline'ı manuel veri parçalama ile yeniden kuruyoruz...")

# --- 2. Veri Yükleme ve MANUEL Parçalama (EN ÖNEMLİ DÜZELTME) ---
try:
    with open("akbank-genai-bootcamp-proje/tarifler.txt", "r", encoding="utf-8") as f:
        tum_tarifler_metni = f.read()

    # Tarifleri '---' ayıracına göre bölüyoruz
    tarif_listesi = tum_tarifler_metni.split("\n---\n")

    # LangChain'in anlayacağı 'Document' formatına çeviriyoruz
    documents = []
    for tarif in tarif_listesi:
        if tarif.strip(): # Boşlukları temizle
            documents.append(Document(page_content=tarif.strip()))

    print(f"\n✅ Veri başarıyla yüklendi ve manuel olarak {len(documents)} adet tarife bölündü.")
    print("(Her bir belge artık tam bir tarifi içeriyor)")

except Exception as e:
    print(f"❌ HATA: tarifler.txt okunurken veya parçalanırken hata: {e}")


# --- 3. Embedding Modelini Yükleme ---
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
print(f"✅ Embedding modeli ({model_name}) yüklendi.")

# --- 4. VEKTÖR VERİTABANINI YENİDEN OLUŞTURMA ---
vector_store = FAISS.from_documents(documents, embeddings)
print("✅ Vektör veritabanı (FAISS) doğru parçalama ile başarıyla oluşturuldu!")

# --- 5. RAG PİPELİNE'INI YENİDEN KURMA ---
llm = ChatGoogleGenerativeAI(model="models/gemini-flash-latest",
                             temperature=0.7,
                             top_p=0.85)

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)
print("✅ RAG Pipeline (Gemini + Manuel Chunking) başarıyla kuruldu.")

# --- 6. RAG PİPELİNE'INI NİHAİ TEST ETME ---
print("\n--- NİHAİ TEST BAŞLIYOR ---")
soru = "Karnıyarık nasıl yapılır? Malzemeleri nelerdir?"
print(f"Soru: {soru}")

try:
    cevap = rag_pipeline.invoke(soru)
    print("\nCevap:")
    print(cevap['result'])
except Exception as e:
    print(f"\n❌ HATA: Model sorgulanamadı. Hata: {e}")

RAG Pipeline'ı manuel veri parçalama ile yeniden kuruyoruz...

✅ Veri başarıyla yüklendi ve manuel olarak 10 adet tarife bölündü.
(Her bir belge artık tam bir tarifi içeriyor)
✅ Embedding modeli (sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2) yüklendi.
✅ Vektör veritabanı (FAISS) doğru parçalama ile başarıyla oluşturuldu!
✅ RAG Pipeline (Gemini + Manuel Chunking) başarıyla kuruldu.

--- NİHAİ TEST BAŞLIYOR ---
Soru: Karnıyarık nasıl yapılır? Malzemeleri nelerdir?

Cevap:
Sağlanan metinler arasında Karnıyarık tarifi bulunmamaktadır.


In [ ]:
# --- 1. Gerekli Import İşlemleri ---
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
import time

print("RAG Pipeline'ı en güçlü 'open source' model ile yeniden kuruyoruz...")
print("Lütfen bekleyin, 'mpnet' modeli (yaklaşık 1.1 GB) indirilecek...")
start_time = time.time()

# --- 2. Veri Yükleme ve MANUEL Parçalama ---
with open("akbank-genai-bootcamp-proje/tarifler.txt", "r", encoding="utf-8") as f:
    tum_tarifler_metni = f.read()

tarif_listesi = tum_tarifler_metni.split("\n---\n")
documents = []
for tarif in tarif_listesi:
    if tarif.strip():
        documents.append(Document(page_content=tarif.strip()))

print(f"\n✅ Veri yüklendi ve {len(documents)} adet tarife bölündü.")

# --- 3. EN GÜÇLÜ EMBEDDING MODELİNİ YÜKLEME ---
# MiniLM yerine, daha büyük ve daha başarılı olan 'mpnet' modelini kullanıyoruz.
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
print(f"✅ Embedding modeli ({model_name}) yüklendi.")

# --- 4. VEKTÖR VERİTABANINI YENİDEN OLUŞTURMA ---
vector_store = FAISS.from_documents(documents, embeddings)
print("✅ Vektör veritabanı (FAISS) 'mpnet' modeli ile başarıyla oluşturuldu!")
end_time = time.time()
print(f"(Bu işlem {end_time - start_time:.2f} saniye sürdü)")

# --- 5. !!! YENİ DEBUG ADIMI: RETRIEVAL KONTROLÜ !!! ---
print("\n--- DEBUG: RETRIEVAL KONTROLÜ BAŞLADI ---")
soru_test = "Karnıyarık tarifi"
try:
    # Veritabanında 'Karnıyarık tarifi'ne en çok benzeyen 1 belgeyi ara
    benzer_belgeler = vector_store.similarity_search(soru_test, k=1)
    if benzer_belgeler:
        print(f"'{soru_test}' sorusuna en çok benzeyen belge:")
        # Çok uzun olmasın diye bulunan belgenin ilk 150 karakterini yazdırıyoruz
        print(benzer_belgeler[0].page_content[:150] + "...")
    else:
        print(f"'{soru_test}' sorusuna benzer belge BULUNAMADI.")
except Exception as e:
    print(f"DEBUG hatası: {e}")
print("--- DEBUG KONTROLÜ BİTTİ ---\n")


# --- 6. RAG PİPELİNE'INI YENİDEN KURMA ---
llm = ChatGoogleGenerativeAI(model="models/gemini-flash-latest",
                             temperature=0.7,
                             top_p=0.85)

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)
print("✅ RAG Pipeline (Gemini + 'mpnet' Embedding) başarıyla kuruldu.")

# --- 7. RAG PİPELİNE'INI NİHAİ TEST ETME ---
print("\n--- NİHAİ TEST (mpnet modeli ile) BAŞLIYOR ---")
soru_final = "Karnıyarık nasıl yapılır? Malzemeleri nelerdir?"
print(f"Soru: {soru_final}")

try:
    cevap = rag_pipeline.invoke(soru_final)
    print("\nCevap:")
    print(cevap['result'])
except Exception as e:
    print(f"❌ HATA: Model sorgulanamadı. Hata: {e}")

RAG Pipeline'ı en güçlü 'open source' model ile yeniden kuruyoruz...
Lütfen bekleyin, 'mpnet' modeli (yaklaşık 1.1 GB) indirilecek...

✅ Veri yüklendi ve 10 adet tarife bölündü.
✅ Embedding modeli (sentence-transformers/paraphrase-multilingual-mpnet-base-v2) yüklendi.
✅ Vektör veritabanı (FAISS) 'mpnet' modeli ile başarıyla oluşturuldu!
(Bu işlem 6.51 saniye sürdü)

--- DEBUG: RETRIEVAL KONTROLÜ BAŞLADI ---
'Karnıyarık tarifi' sorusuna en çok benzeyen belge:
Başlık: İçli Köfte
Malzemeler:
Dış Harcı İçin:
- 2 su bardağı ince bulgur (köftelik)
- 1 su bardağı irmik
- 1 adet yumurta
- 1 yemek kaşığı biber salç...
--- DEBUG KONTROLÜ BİTTİ ---

✅ RAG Pipeline (Gemini + 'mpnet' Embedding) başarıyla kuruldu.

--- NİHAİ TEST (mpnet modeli ile) BAŞLIYOR ---
Soru: Karnıyarık nasıl yapılır? Malzemeleri nelerdir?

Cevap:
Bu bilgiye göre Karnıyarık tarifini ve malzemelerini bilmiyorum.


In [ ]:
# --- Adım 1: Eski Klonu Silme ---
!rm -rf akbank-genai-bootcamp-proje
print("Eski repo kopyası silindi.")

# --- Adım 2: Reponun Güncel Halini Yeniden Klonlama ---
!git clone https://github.com/mustafadevrim/akbank-genai-bootcamp-proje
print("\nReponun güncel hali klonlandı.")

# --- Adım 3: 'tarifler.txt' Dosyasının İçeriğini Kontrol Etme ---
print("\n--- GÜNCEL DOSYA İÇERİK KONTROLÜ ---")
try:
    with open("akbank-genai-bootcamp-proje/tarifler.txt", "r", encoding="utf-8") as f:
        icerik = f.read()

    if "Karnıyarık" in icerik and "Menemen" in icerik:
        print("✅ BAŞARILI: 'Karnıyarık' ve 'Menemen' tarifleri güncel dosyada bulundu!")
        print("Şimdi RAG pipeline'ını tekrar kurabilirsiniz.")
    else:
        print("❌ KRİTİK HATA: 'Karnıyarık' tarifi dosyada hala bulunamıyor.")
        print("Lütfen GitHub reponuza 'tarifler.txt' dosyasını (10 tarifin tamamını) eklediğinizden emin olun.")

except Exception as e:
    print(f"HATA: Dosya okunamadı. {e}")

Eski repo kopyası silindi.
Cloning into 'akbank-genai-bootcamp-proje'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 11.68 KiB | 3.89 MiB/s, done.
Resolving deltas: 100% (2/2), done.

Reponun güncel hali klonlandı.

--- GÜNCEL DOSYA İÇERİK KONTROLÜ ---
✅ BAŞARILI: 'Karnıyarık' ve 'Menemen' tarifleri güncel dosyada bulundu!
Şimdi RAG pipeline'ını tekrar kurabilirsiniz.


In [ ]:
import sys

# === ADIM 1: KURULUM ===
print("--- ADIM 1: Kütüphaneler kuruluyor... ---")
# Proje dosyasının istediği tüm teknolojileri kur
!pip install -q -U google-generativeai langchain-google-genai langchain faiss-cpu langchain_community sentence-transformers
print("✅ Kütüphaneler kuruldu.")

# === ADIM 2: API ANAHTARI VE IMPORTLAR ===
print("\n--- ADIM 2: Kütüphaneler ve API Anahtarı yükleniyor... ---")
import os
from google.colab import userdata
import google.generativeai as genai
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
import time

try:
    # Colab Secrets (🔑) bölümünden API anahtarını al
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    print("✅ API Anahtarı ve Kütüphaneler Başarıyla Yüklendi!")
except Exception as e:
    print(f"❌ HATA: API Anahtarı yüklenemedi. 'Secrets' (🔑) bölümünü kontrol edin. Hata: {e}")
    # API Anahtarı olmadan devam etme
    sys.exit("API Anahtarı hatası. Lütfen 'Secrets' bölümünü kontrol edin.")

# === ADIM 3: GÜNCEL VERİYİ ÇEKME ===
print("\n--- ADIM 3: GitHub'dan güncel veri seti çekiliyor... ---")
# Önce eski, bozuk klonu sil
!rm -rf akbank-genai-bootcamp-proje
# Güncel repoyu klonla
!git clone https://github.com/mustafadevrim/akbank-genai-bootcamp-proje
print("✅ Güncel repo klonlandı.")

# === ADIM 4: VERİ YÜKLEME VE MANUEL PARÇALAMA ===
print("\n--- ADIM 4: Veri yükleniyor ve manuel parçalanıyor... ---")
try:
    with open("akbank-genai-bootcamp-proje/tarifler.txt", "r", encoding="utf-8") as f:
        tum_tarifler_metni = f.read()

    # Tarifleri '---' ayıracına göre böl
    tarif_listesi = tum_tarifler_metni.split("\n---\n")
    documents = []
    for tarif in tarif_listesi:
        if tarif.strip():
            documents.append(Document(page_content=tarif.strip()))

    print(f"✅ Veri başarıyla yüklendi ve {len(documents)} adet tarife bölündü.")
    # Karnıyarık'ın dosyada olduğunu bir daha kontrol et
    if "Karnıyarık" not in tum_tarifler_metni:
        print("❌ UYARI: 'Karnıyarık' metni dosyada bulunamadı. Lütfen GitHub'ı kontrol edin.")

except Exception as e:
    print(f"❌ HATA: tarifler.txt okunurken veya parçalanırken hata: {e}")
    sys.exit("Veri yükleme hatası.")

# === ADIM 5: EMBEDDING VE VEKTÖR DB ===
print("\n--- ADIM 5: Embedding modeli yükleniyor ve Vektör DB oluşturuluyor... ---")
print("Lütfen bekleyin, 'mpnet' modeli (yaklaşık 1.1 GB) indirilecek...")
start_time = time.time()

# Proje dosyasında izin verilen "open source" model
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
print(f"✅ Embedding modeli ({model_name}) yüklendi.")

# Temiz 'documents' listesi ile Vektör DB'yi oluştur
vector_store = FAISS.from_documents(documents, embeddings)
print("✅ Vektör veritabanı (FAISS) 'mpnet' modeli ile başarıyla oluşturuldu!")
end_time = time.time()
print(f"(Bu işlem {end_time - start_time:.2f} saniye sürdü)")

# === ADIM 6: DEBUG KONTROLÜ ===
print("\n--- ADIM 6: DEBUG - RETRIEVAL KONTROLÜ ---")
soru_test = "Karnıyarık tarifi"
try:
    # Vektör DB'de arama yap
    benzer_belgeler = vector_store.similarity_search(soru_test, k=1)

    # Dönen belgenin içeriğini kontrol et
    if benzer_belgeler and "Karnıyarık" in benzer_belgeler[0].page_content:
        print(f"✅ DEBUG BAŞARILI: '{soru_test}' sorusuna en çok benzeyen belge:")
        print(benzer_blog = benzer_belgeler[0].page_content[:150] + "...")
    else:
        print(f"❌ DEBUG BAŞARISIZ: '{soru_test}' sorusuna benzer belge 'Karnıyarık' değil veya bulunamadı.")
        if benzer_belgeler:
             print("Dönen belge:", benzer_belgeler[0].page_content[:150] + "...")
        else:
            print("Hiç belge dönemedi.")

except Exception as e:
    print(f"❌ DEBUG hatası: {e}")
print("--- DEBUG KONTROLÜ BİTTİ ---\n")

# === ADIM 7: RAG PIPELINE VE NİHAİ TEST ===
print("--- ADIM 7: RAG Pipeline kuruluyor ve test ediliyor... ---")

# Proje dosyasında izin verilen "Generation model"
llm = ChatGoogleGenerativeAI(model="models/gemini-flash-latest",
                             temperature=0.7,
                             top_p=0.85)

# Proje dosyasında izin verilen "RAG pipeline framework" (LangChain)
rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)
print("✅ RAG Pipeline (Gemini + 'mpnet' Embedding) başarıyla kuruldu.")

print("\n--- NİHAİ TEST (mpnet modeli ile) BAŞLIYOR ---")
soru_final = "Karnıyarık nasıl yapılır? Malzemeleri nelerdir?"
print(f"Soru: {soru_final}")

try:
    cevap = rag_pipeline.invoke(soru_final)
    print("\nCevap:")
    print(cevap['result'])
except Exception as e:
    print(f"❌ HATA: Model sorgulanamadı. Hata: {e}")

--- ADIM 1: Kütüphaneler kuruluyor... ---
✅ Kütüphaneler kuruldu.

--- ADIM 2: Kütüphaneler ve API Anahtarı yükleniyor... ---
✅ API Anahtarı ve Kütüphaneler Başarıyla Yüklendi!

--- ADIM 3: GitHub'dan güncel veri seti çekiliyor... ---
Cloning into 'akbank-genai-bootcamp-proje'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 11.68 KiB | 11.68 MiB/s, done.
Resolving deltas: 100% (2/2), done.
✅ Güncel repo klonlandı.

--- ADIM 4: Veri yükleniyor ve manuel parçalanıyor... ---
✅ Veri başarıyla yüklendi ve 10 adet tarife bölündü.

--- ADIM 5: Embedding modeli yükleniyor ve Vektör DB oluşturuluyor... ---
Lütfen bekleyin, 'mpnet' modeli (yaklaşık 1.1 GB) indirilecek...


/tmp/ipython-input-1347041822.py:68: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Embedding modeli (sentence-transformers/paraphrase-multilingual-mpnet-base-v2) yüklendi.
✅ Vektör veritabanı (FAISS) 'mpnet' modeli ile başarıyla oluşturuldu!
(Bu işlem 21.51 saniye sürdü)

--- ADIM 6: DEBUG - RETRIEVAL KONTROLÜ ---
❌ DEBUG BAŞARISIZ: 'Karnıyarık tarifi' sorusuna benzer belge 'Karnıyarık' değil veya bulunamadı.
Dönen belge: Başlık: İçli Köfte
Malzemeler:
Dış Harcı İçin:
- 2 su bardağı ince bulgur (köftelik)
- 1 su bardağı irmik
- 1 adet yumurta
- 1 yemek kaşığı biber salç...
--- DEBUG KONTROLÜ BİTTİ ---

--- ADIM 7: RAG Pipeline kuruluyor ve test ediliyor... ---
✅ RAG Pipeline (Gemini + 'mpnet' Embedding) başarıyla kuruldu.

--- NİHAİ TEST (mpnet modeli ile) BAŞLIYOR ---
Soru: Karnıyarık nasıl yapılır? Malzemeleri nelerdir?

Cevap:
Sağlanan metinlerde Karnıyarık tarifi bulunmamaktadır. Yalnızca benzer bir yemek olan İmam Bayıldı'nın tarifi mevcuttur.


In [ ]:
import sys
import time

# === ADIM 1: KURULUM ===
print("--- ADIM 1: Kütüphaneler kuruluyor... ---")
!pip install -q -U google-generativeai langchain-google-genai langchain faiss-cpu langchain_community sentence-transformers
print("✅ Kütüphaneler kuruldu.")

# === ADIM 2: API ANAHTARI VE IMPORTLAR ===
print("\n--- ADIM 2: Kütüphaneler ve API Anahtarı yükleniyor... ---")
import os
from google.colab import userdata
import google.generativeai as genai
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
# YENİ: RecursiveCharacterTextSplitter'ı tekrar kullanacağız, ama daha akıllıca.
from langchain.text_splitter import RecursiveCharacterTextSplitter

try:
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    print("✅ API Anahtarı ve Kütüphaneler Başarıyla Yüklendi!")
except Exception as e:
    print(f"❌ HATA: API Anahtarı yüklenemedi. 'Secrets' (🔑) bölümünü kontrol edin. Hata: {e}")
    sys.exit("API Anahtarı hatası.")

# === ADIM 3: GÜNCEL VERİYİ ÇEKME ===
print("\n--- ADIM 3: GitHub'dan güncel veri seti çekiliyor... ---")
!rm -rf akbank-genai-bootcamp-proje
!git clone https://github.com/mustafadevrim/akbank-genai-bootcamp-proje
print("✅ Güncel repo klonlandı.")

# === ADIM 4: YENİ SEMANTİK PARÇALAMA (STRATEJİ DEĞİŞİKLİĞİ) ===
print("\n--- ADIM 4: Veri yükleniyor ve SEMANTİK olarak parçalanıyor... ---")
try:
    with open("akbank-genai-bootcamp-proje/tarifler.txt", "r", encoding="utf-8") as f:
        tum_tarifler_metni = f.read()

    # ÖNCE: Tarifleri '---' ayıracına göre ayır
    tarif_listesi = tum_tarifler_metni.split("\n---\n")
    print(f"{len(tarif_listesi)} adet ana tarif bulundu.")

    # SONRA: Her tarifi kendi içinde semantik olarak bölmek için bir ayırıcı tanımla
    text_splitter = RecursiveCharacterTextSplitter(
        # Bu ayırıcılar tarif yapımıza özel:
        separators=["Başlık: ", "\nMalzemeler:\n", "\nYapılışı:\n", "\n\n", "\n"],
        chunk_size=500, # Chunklar daha küçük ve odaklı olsun
        chunk_overlap=50
    )

    all_chunks = []
    # Her bir tarifi (10 adet) alıp, kendi içinde parçalıyoruz
    for tarif_metni in tarif_listesi:
        if tarif_metni.strip():
            # 'create_documents' metni bölümlere ayırır
            chunks = text_splitter.create_documents([tarif_metni.strip()])
            all_chunks.extend(chunks)

    print(f"✅ Veri başarıyla {len(all_chunks)} adet küçük semantik parçaya (chunk) bölündü.")

except Exception as e:
    print(f"❌ HATA: Veri yükleme veya parçalama hatası: {e}")
    sys.exit("Veri yükleme hatası.")

# === ADIM 5: EMBEDDING VE VEKTÖR DB ===
print("\n--- ADIM 5: Embedding modeli yükleniyor ve Vektör DB oluşturuluyor... ---")
print("Lütfen bekleyin, 'mpnet' modeli (yaklaşık 1.1 GB) indirilecek...")
start_time = time.time()

model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
print(f"✅ Embedding modeli ({model_name}) yüklendi.")

# Vektör DB'yi küçük parçalarla (all_chunks) oluştur
vector_store = FAISS.from_documents(all_chunks, embeddings)
print("✅ Vektör veritabanı (FAISS) 'mpnet' ve semantik chunking ile başarıyla oluşturuldu!")
end_time = time.time()
print(f"(Bu işlem {end_time - start_time:.2f} saniye sürdü)")

# === ADIM 6: DEBUG KONTROLÜ ===
print("\n--- ADIM 6: DEBUG - RETRIEVAL KONTROLÜ ---")
soru_test = "Karnıyarık tarifi"
try:
    # Vektör DB'de arama yap
    benzer_belgeler = vector_store.similarity_search(soru_test, k=1)

    if benzer_belgeler and "Karnıyarık" in benzer_belgeler[0].page_content:
        print(f"✅ DEBUG BAŞARILI: '{soru_test}' sorusuna en çok benzeyen belge:")
        print(benzer_belgeler[0].page_content[:150] + "...")
    else:
        print(f"❌ DEBUG BAŞARISIZ: '{soru_test}' sorusuna benzer belge 'Karnıyarık' değil veya bulunamadı.")
        if benzer_belgeler:
             print("Dönen belge:", benzer_belgeler[0].page_content[:150] + "...")
        else:
            print("Hiç belge dönemedi.")

except Exception as e:
    print(f"❌ DEBUG hatası: {e}")
print("--- DEBUG KONTROLÜ BİTTİ ---\n")

# === ADIM 7: RAG PIPELINE VE NİHAİ TEST ===
print("--- ADIM 7: RAG Pipeline kuruluyor ve test ediliyor... ---")

llm = ChatGoogleGenerativeAI(model="models/gemini-flash-latest",
                             temperature=0.7,
                             top_p=0.85)

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)
print("✅ RAG Pipeline (Gemini + 'mpnet' + Semantik Chunking) başarıyla kuruldu.")

print("\n--- NİHAİ TEST (Semantik Parçalama ile) BAŞLIYOR ---")
soru_final = "Karnıyarık nasıl yapılır? Malzemeleri nelerdir?"
print(f"Soru: {soru_final}")

try:
    cevap = rag_pipeline.invoke(soru_final)
    print("\nCevap:")
    print(cevap['result'])
except Exception as e:
    print(f"❌ HATA: Model sorgulanamadı. Hata: {e}")

--- ADIM 1: Kütüphaneler kuruluyor... ---
✅ Kütüphaneler kuruldu.

--- ADIM 2: Kütüphaneler ve API Anahtarı yükleniyor... ---
✅ API Anahtarı ve Kütüphaneler Başarıyla Yüklendi!

--- ADIM 3: GitHub'dan güncel veri seti çekiliyor... ---
Cloning into 'akbank-genai-bootcamp-proje'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 11.68 KiB | 5.84 MiB/s, done.
Resolving deltas: 100% (2/2), done.
✅ Güncel repo klonlandı.

--- ADIM 4: Veri yükleniyor ve SEMANTİK olarak parçalanıyor... ---
10 adet ana tarif bulundu.
✅ Veri başarıyla 45 adet küçük semantik parçaya (chunk) bölündü.

--- ADIM 5: Embedding modeli yükleniyor ve Vektör DB oluşturuluyor... ---
Lütfen bekleyin, 'mpnet' modeli (yaklaşık 1.1 GB) indirilecek...
✅ Embedding modeli (sentence-transformers/paraphrase-multilingual-mpnet-base-v2) yüklendi.

In [ ]:
# --- ADIM 6 (REVİZE EDİLDİ): DEBUG KONTROLÜ (k=5) ---
print("\n--- ADIM 6: DEBUG - RETRIEVAL KONTROLÜ (k=5) ---")
print("Karnıyarık'a benzeyen EN İYİ 5 parçayı (chunk) arıyoruz:")
soru_test = "Karnıyarık tarifi"
try:
    # k=1 yerine k=5 (en iyi 5) arıyoruz
    benzer_belgeler = vector_store.similarity_search(soru_test, k=5)

    if benzer_belgeler:
        for i, doc in enumerate(benzer_belgeler):
            print(f"\n--- DÖNEN BELGE {i+1} ('Karnıyarık' içermeli) ---")
            print(doc.page_content[:150] + "...")
    else:
        print("Hiç belge dönemedi.")

except Exception as e:
    print(f"❌ DEBUG hatası: {e}")
print("\n--- DEBUG KONTROLÜ BİTTİ ---\n")


# --- ADIM 7 (REVİZE EDİLDİ): RAG PIPELINE (k=5) ---
print("--- ADIM 7: RAG Pipeline (k=5 ile) kuruluyor ve test ediliyor... ---")

llm = ChatGoogleGenerativeAI(model="models/gemini-flash-latest",
                             temperature=0.7,
                             top_p=0.85)

# !!! EN ÖNEMLİ DEĞİŞİKLİK BURADA !!!
# Retriever'a "sadece 1 değil, 5 belge getir" (k=5) diyoruz.
# Bu sayede hem 'Başlık' hem 'Malzemeler' hem de 'Yapılışı' parçaları gelecektir.
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever # Düzeltilmiş (k=5) retriever'ı kullan
)
print("✅ RAG Pipeline (k=5 ile) başarıyla kuruldu.")

print("\n--- NİHAİ TEST (k=5 ile) BAŞLIYOR ---")
soru_final = "Karnıyarık nasıl yapılır? Malzemeleri nelerdir?"
print(f"Soru: {soru_final}")

try:
    cevap = rag_pipeline.invoke(soru_final)
    print("\nCevap:")
    print(cevap['result'])
except Exception as e:
    print(f"❌ HATA: Model sorgulanamadı. Hata: {e}")


--- ADIM 6: DEBUG - RETRIEVAL KONTROLÜ (k=5) ---
Karnıyarık'a benzeyen EN İYİ 5 parçayı (chunk) arıyoruz:

--- DÖNEN BELGE 1 ('Karnıyarık' içermeli) ---
Başlık: Karnıyarık...

--- DÖNEN BELGE 2 ('Karnıyarık' içermeli) ---
Başlık: İçli Köfte...

--- DÖNEN BELGE 3 ('Karnıyarık' içermeli) ---
Başlık: Künefe...

--- DÖNEN BELGE 4 ('Karnıyarık' içermeli) ---
Başlık: Zeytinyağlı Yaprak Sarma...

--- DÖNEN BELGE 5 ('Karnıyarık' içermeli) ---
Başlık: Kuru Fasulye...

--- DEBUG KONTROLÜ BİTTİ ---

--- ADIM 7: RAG Pipeline (k=5 ile) kuruluyor ve test ediliyor... ---
✅ RAG Pipeline (k=5 ile) başarıyla kuruldu.

--- NİHAİ TEST (k=5 ile) BAŞLIYOR ---
Soru: Karnıyarık nasıl yapılır? Malzemeleri nelerdir?

Cevap:
Verilen metin parçalarında Karnıyarık'ın yapılışına veya malzemelerine dair bilgi bulunmamaktadır.


In [ ]:
import sys
import time

# === ADIM 1: KURULUM ===
print("--- ADIM 1: Kütüphaneler kuruluyor... ---")
!pip install -q -U google-generativeai langchain-google-genai langchain faiss-cpu langchain_community sentence-transformers
print("✅ Kütüphaneler kuruldu.")

# === ADIM 2: API ANAHTARI VE IMPORTLAR ===
print("\n--- ADIM 2: Kütüphaneler ve API Anahtarı yükleniyor... ---")
import os
from google.colab import userdata
import google.generativeai as genai
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document

try:
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    print("✅ API Anahtarı ve Kütüphaneler Başarıyla Yüklendi!")
except Exception as e:
    print(f"❌ HATA: API Anahtarı yüklenemedi. 'Secrets' (🔑) bölümünü kontrol edin. Hata: {e}")
    sys.exit("API Anahtarı hatası.")

# === ADIM 3: GÜNCEL VERİYİ ÇEKME ===
print("\n--- ADIM 3: GitHub'dan güncel veri seti çekiliyor... ---")
!rm -rf akbank-genai-bootcamp-proje
!git clone https://github.com/mustafadevrim/akbank-genai-bootcamp-proje
print("✅ Güncel repo klonlandı.")

# === ADIM 4: YENİ KONTEKST ENJEKSİYONU PARÇALAMASI ===
print("\n--- ADIM 4: Veri yükleniyor ve KONTEKST ENJEKSİYONU ile parçalanıyor... ---")
try:
    with open("akbank-genai-bootcamp-proje/tarifler.txt", "r", encoding="utf-8") as f:
        tum_tarifler_metni = f.read()

    tarif_listesi = tum_tarifler_metni.split("\n---\n")
    print(f"{len(tarif_listesi)} adet ana tarif bulundu.")

    documents = [] # Parçaları (chunk) tutacak son liste

    for tarif_metni in tarif_listesi:
        if not tarif_metni.strip():
            continue

        # 1. Başlığı ayır (İlk satır)
        parts = tarif_metni.split("\nMalzemeler:\n", 1)
        if len(parts) < 2: continue # Bozuk format, atla

        baslik_content = parts[0].strip() # "Başlık: Karnıyarık"

        # 2. Malzemeler ve Yapılışı'nı ayır
        parts2 = parts[1].split("\nYapılışı:\n", 1)
        if len(parts2) < 2: continue # Bozuk format, atla

        malzemeler_content = parts2[0].strip() # "- 6 adet patlıcan..."
        yapilis_content = parts2[1].strip()   # "1. Patlıcanları soyun..."

        # 3. Konteksi enjekte ederek 3 ayrı 'Document' (chunk) oluştur

        # Chunk 1: Sadece Başlık (Aramada bunu bulması için)
        documents.append(Document(
            page_content=baslik_content,
            metadata={"source": baslik_content} # Metadata'ya da başlığı ekle
        ))

        # Chunk 2: Başlık + Malzemeler (Artık 'Karnıyarık' bilgisi içeriyor)
        documents.append(Document(
            page_content=f"{baslik_content}\nMalzemeler:\n{malzemeler_content}",
            metadata={"source": baslik_content}
        ))

        # Chunk 3: Başlık + Yapılışı (Artık 'Karnıyarık' bilgisi içeriyor)
        documents.append(Document(
            page_content=f"{baslik_content}\nYapılışı:\n{yapilis_content}",
            metadata={"source": baslik_content}
        ))

    print(f"✅ Veri başarıyla {len(documents)} adet KONTEKSTLİ parçaya (chunk) bölündü. (10 tarif x 3 parça = 30)")

except Exception as e:
    print(f"❌ HATA: Veri yükleme veya parçalama hatası: {e}")
    sys.exit("Veri yükleme hatası.")

# === ADIM 5: EMBEDDING VE VEKTÖR DB ===
print("\n--- ADIM 5: Embedding modeli yükleniyor ve Vektör DB oluşturuluyor... ---")
print("Lütfen bekleyin, 'mpnet' modeli (yaklaşık 1.1 GB) indirilecek...")
start_time = time.time()

model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
print(f"✅ Embedding modeli ({model_name}) yüklendi.")

# Vektör DB'yi KONTEKSTLİ parçalarla (documents) oluştur
vector_store = FAISS.from_documents(documents, embeddings)
print("✅ Vektör veritabanı (FAISS) 'mpnet' ve 'Konteks Enjeksiyonu' ile başarıyla oluşturuldu!")
end_time = time.time()
print(f"(Bu işlem {end_time - start_time:.2f} saniye sürdü)")

# === ADIM 6: DEBUG KONTROLÜ (k=3) ===
print("\n--- ADIM 6: DEBUG - RETRIEVAL KONTROLÜ (k=3) ---")
print("Karnıyarık'a benzeyen EN İYİ 3 parçayı (chunk) arıyoruz:")
soru_test = "Karnıyarık tarifi"
try:
    # k=3 arıyoruz (Başlık, Malzemeler, Yapılış)
    benzer_belgeler = vector_store.similarity_search(soru_test, k=3)

    if benzer_belgeler:
        for i, doc in enumerate(benzer_belgeler):
            print(f"\n--- DÖNEN BELGE {i+1} (MUTLAKA 'Karnıyarık' içermeli) ---")
            print(doc.page_content[:150] + "...")
    else:
        print("Hiç belge dönemedi.")

except Exception as e:
    print(f"❌ DEBUG hatası: {e}")
print("\n--- DEBUG KONTROLÜ BİTTİ ---\n")

# === ADIM 7: RAG PIPELINE VE NİHAİ TEST (k=3) ===
print("--- ADIM 7: RAG Pipeline (k=3 ile) kuruluyor ve test ediliyor... ---")

llm = ChatGoogleGenerativeAI(model="models/gemini-flash-latest",
                             temperature=0.7,
                             top_p=0.85)

# Retriever'a "en iyi 3 belgeyi getir" diyoruz (k=3)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever # Düzeltilmiş (k=3) retriever'ı kullan
)
print("✅ RAG Pipeline (k=3 ile) başarıyla kuruldu.")

print("\n--- NİHAİ TEST (Konteks Enjeksiyonu ile) BAŞLIYOR ---")
soru_final = "Karnıyarık nasıl yapılır? Malzemeleri nelerdir?"
print(f"Soru: {soru_final}")

try:
    cevap = rag_pipeline.invoke(soru_final)
    print("\nCevap:")
    print(cevap['result'])
except Exception as e:
    print(f"❌ HATA: Model sorgulanamadı. Hata: {e}")

--- ADIM 1: Kütüphaneler kuruluyor... ---
✅ Kütüphaneler kuruldu.

--- ADIM 2: Kütüphaneler ve API Anahtarı yükleniyor... ---
✅ API Anahtarı ve Kütüphaneler Başarıyla Yüklendi!

--- ADIM 3: GitHub'dan güncel veri seti çekiliyor... ---
Cloning into 'akbank-genai-bootcamp-proje'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
Receiving objects: 100% (14/14), 11.68 KiB | 11.68 MiB/s, done.
Resolving deltas: 100% (2/2), done.
remote: Total 14 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
✅ Güncel repo klonlandı.

--- ADIM 4: Veri yükleniyor ve KONTEKST ENJEKSİYONU ile parçalanıyor... ---
10 adet ana tarif bulundu.
✅ Veri başarıyla 30 adet KONTEKSTLİ parçaya (chunk) bölündü. (10 tarif x 3 parça = 30)

--- ADIM 5: Embedding modeli yükleniyor ve Vektör DB oluşturuluyor... ---
Lütfen bekleyin, 'mpnet' modeli (yaklaşık 1.1 GB) indirilecek...
✅ Embedding modeli (sentence-transformers/paraphrase-multi

In [6]:
import sys
import time

# === ADIM 1: KURULUM ===
print("--- ADIM 1: Kütüphaneler kuruluyor... ---")
!pip install -q -U google-generativeai langchain-google-genai langchain faiss-cpu langchain_community sentence-transformers
print("✅ Kütüphaneler kuruldu.")

# === ADIM 2: API ANAHTARI VE IMPORTLAR ===
print("\n--- ADIM 2: Kütüphaneler ve API Anahtarı yükleniyor... ---")
import os
from google.colab import userdata
import google.generativeai as genai
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document

try:
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    print("✅ API Anahtarı ve Kütüphaneler Başarıyla Yüklendi!")
except Exception as e:
    print(f"❌ HATA: API Anahtarı yüklenemedi. 'Secrets' (🔑) bölümünü kontrol edin. Hata: {e}")
    sys.exit("API Anahtarı hatası.")

# === ADIM 3: GÜNCEL VERİYİ ÇEKME ===
print("\n--- ADIM 3: GitHub'dan güncel veri seti çekiliyor... ---")
!rm -rf akbank-genai-bootcamp-proje
!git clone https://github.com/mustafadevrim/akbank-genai-bootcamp-proje
print("✅ Güncel repo klonlandı.")

# === ADIM 4: YENİ KONTEKST ENJEKSİYONU PARÇALAMASI ===
print("\n--- ADIM 4: Veri yükleniyor ve KONTEKST ENJEKSİYONU ile parçalanıyor... ---")
try:
    with open("akbank-genai-bootcamp-proje/tarifler.txt", "r", encoding="utf-8") as f:
        tum_tarifler_metni = f.read()

    tarif_listesi = tum_tarifler_metni.split("\n---\n")
    print(f"{len(tarif_listesi)} adet ana tarif bulundu.")

    documents = [] # Parçaları (chunk) tutacak son liste

    for tarif_metni in tarif_listesi:
        if not tarif_metni.strip():
            continue

        # 1. Başlığı ayır (İlk satır)
        parts = tarif_metni.split("\nMalzemeler:\n", 1)
        if len(parts) < 2: continue # Bozuk format, atla

        baslik_content = parts[0].strip() # "Başlık: Karnıyarık"

        # 2. Malzemeler ve Yapılışı'nı ayır
        parts2 = parts[1].split("\nYapılışı:\n", 1)
        if len(parts2) < 2: continue # Bozuk format, atla

        malzemeler_content = parts2[0].strip() # "- 6 adet patlıcan..."
        yapilis_content = parts2[1].strip()   # "1. Patlıcanları soyun..."

        # 3. Konteksi enjekte ederek 3 ayrı 'Document' (chunk) oluştur

        # Chunk 1: Sadece Başlık (Aramada bunu bulması için)
        documents.append(Document(
            page_content=baslik_content,
            metadata={"source": baslik_content} # Metadata'ya da başlığı ekle
        ))

        # Chunk 2: Başlık + Malzemeler (Artık 'Karnıyarık' bilgisi içeriyor)
        documents.append(Document(
            page_content=f"{baslik_content}\nMalzemeler:\n{malzemeler_content}",
            metadata={"source": baslik_content}
        ))

        # Chunk 3: Başlık + Yapılışı (Artık 'Karnıyarık' bilgisi içeriyor)
        documents.append(Document(
            page_content=f"{baslik_content}\nYapılışı:\n{yapilis_content}",
            metadata={"source": baslik_content}
        ))

    print(f"✅ Veri başarıyla {len(documents)} adet KONTEKSTLİ parçaya (chunk) bölündü. (10 tarif x 3 parça = 30)")

except Exception as e:
    print(f"❌ HATA: Veri yükleme veya parçalama hatası: {e}")
    sys.exit("Veri yükleme hatası.")

# === ADIM 5: EMBEDDING VE VEKTÖR DB ===
print("\n--- ADIM 5: Embedding modeli yükleniyor ve Vektör DB oluşturuluyor... ---")
print("Lütfen bekleyin, 'mpnet' modeli (yaklaşık 1.1 GB) indirilecek...")
start_time = time.time()

model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
print(f"✅ Embedding modeli ({model_name}) yüklendi.")

# Vektör DB'yi KONTEKSTLİ parçalarla (documents) oluştur
vector_store = FAISS.from_documents(documents, embeddings)
print("✅ Vektör veritabanı (FAISS) 'mpnet' ve 'Konteks Enjeksiyonu' ile başarıyla oluşturuldu!")
end_time = time.time()
print(f"(Bu işlem {end_time - start_time:.2f} saniye sürdü)")

# === ADIM 6: DEBUG KONTROLÜ (k=3) ===
print("\n--- ADIM 6: DEBUG - RETRIEVAL KONTROLÜ (k=3) ---")
print("Karnıyarık'a benzeyen EN İYİ 3 parçayı (chunk) arıyoruz:")
soru_test = "Karnıyarık tarifi"
try:
    # k=3 arıyoruz (Başlık, Malzemeler, Yapılış)
    benzer_belgeler = vector_store.similarity_search(soru_test, k=3)

    if benzer_belgeler:
        for i, doc in enumerate(benzer_belgeler):
            print(f"\n--- DÖNEN BELGE {i+1} (MUTLAKA 'Karnıyarık' içermeli) ---")
            print(doc.page_content[:150] + "...")
    else:
        print("Hiç belge dönemedi.")

except Exception as e:
    print(f"❌ DEBUG hatası: {e}")
print("\n--- DEBUG KONTROLÜ BİTTİ ---\n")

# === ADIM 7: RAG PIPELINE VE NİHAİ TEST (k=3) ===
print("--- ADIM 7: RAG Pipeline (k=3 ile) kuruluyor ve test ediliyor... ---")

llm = ChatGoogleGenerativeAI(model="models/gemini-flash-latest",
                             temperature=0.7,
                             top_p=0.85)

# Retriever'a "en iyi 3 belgeyi getir" diyoruz (k=3)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever # Düzeltilmiş (k=3) retriever'ı kullan
)
print("✅ RAG Pipeline (k=3 ile) başarıyla kuruldu.")

print("\n--- NİHAİ TEST (Konteks Enjeksiyonu ile) BAŞLIYOR ---")
soru_final = "Karnıyarık nasıl yapılır? Malzemeleri nelerdir?"
print(f"Soru: {soru_final}")

try:
    cevap = rag_pipeline.invoke(soru_final)
    print("\nCevap:")
    print(cevap['result'])
except Exception as e:
    print(f"❌ HATA: Model sorgulanamadı. Hata: {e}")

--- ADIM 1: Kütüphaneler kuruluyor... ---
✅ Kütüphaneler kuruldu.

--- ADIM 2: Kütüphaneler ve API Anahtarı yükleniyor... ---
✅ API Anahtarı ve Kütüphaneler Başarıyla Yüklendi!

--- ADIM 3: GitHub'dan güncel veri seti çekiliyor... ---
Cloning into 'akbank-genai-bootcamp-proje'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 11.68 KiB | 11.68 MiB/s, done.
Resolving deltas: 100% (2/2), done.
✅ Güncel repo klonlandı.

--- ADIM 4: Veri yükleniyor ve KONTEKST ENJEKSİYONU ile parçalanıyor... ---
10 adet ana tarif bulundu.
✅ Veri başarıyla 30 adet KONTEKSTLİ parçaya (chunk) bölündü. (10 tarif x 3 parça = 30)

--- ADIM 5: Embedding modeli yükleniyor ve Vektör DB oluşturuluyor... ---
Lütfen bekleyin, 'mpnet' modeli (yaklaşık 1.1 GB) indirilecek...
✅ Embedding modeli (sentence-transformers/paraphrase-multi

In [1]:
import sys
import time

# === ADIM 1: KURULUM ===
print("--- ADIM 1: Kütüphaneler kuruluyor... ---")
!pip install -q -U google-generativeai langchain-google-genai langchain faiss-cpu langchain_community sentence-transformers
print("✅ Kütüphaneler kuruldu.")

# === ADIM 2: API ANAHTARI VE IMPORTLAR ===
print("\n--- ADIM 2: Kütüphaneler ve API Anahtarı yükleniyor... ---")
import os
from google.colab import userdata
import google.generativeai as genai
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document

try:
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    print("✅ API Anahtarı ve Kütüphaneler Başarıyla Yüklendi!")
except Exception as e:
    print(f"❌ HATA: API Anahtarı yüklenemedi. 'Secrets' (🔑) bölümünü kontrol edin. Hata: {e}")
    sys.exit("API Anahtarı hatası.")

# === ADIM 3: GÜNCEL VERİYİ ÇEKME ===
print("\n--- ADIM 3: GitHub'dan güncel veri seti çekiliyor... ---")
!rm -rf akbank-genai-bootcamp-proje
!git clone https://github.com/mustafadevrim/akbank-genai-bootcamp-proje
print("✅ Güncel repo klonlandı.")

# === ADIM 4: YENİ KONTEKST ENJEKSİYONU PARÇALAMASI ===
print("\n--- ADIM 4: Veri yükleniyor ve KONTEKST ENJEKSİYONU ile parçalanıyor... ---")
try:
    with open("akbank-genai-bootcamp-proje/tarifler.txt", "r", encoding="utf-8") as f:
        tum_tarifler_metni = f.read()

    tarif_listesi = tum_tarifler_metni.split("\n---\n")
    print(f"{len(tarif_listesi)} adet ana tarif bulundu.")

    documents = [] # Parçaları (chunk) tutacak son liste

    for tarif_metni in tarif_listesi:
        if not tarif_metni.strip(): continue
        parts = tarif_metni.split("\nMalzemeler:\n", 1)
        if len(parts) < 2: continue
        baslik_content = parts[0].strip()
        parts2 = parts[1].split("\nYapılışı:\n", 1)
        if len(parts2) < 2: continue
        malzemeler_content = parts2[0].strip()
        yapilis_content = parts2[1].strip()

        # Stratejimiz (3 parça oluşturma) aynı
        documents.append(Document(page_content=baslik_content, metadata={"source": baslik_content}))
        documents.append(Document(page_content=f"{baslik_content}\nMalzemeler:\n{malzemeler_content}", metadata={"source": baslik_content}))
        documents.append(Document(page_content=f"{baslik_content}\nYapılışı:\n{yapilis_content}", metadata={"source": baslik_content}))

    print(f"✅ Veri başarıyla {len(documents)} adet KONTEKSTLİ parçaya (chunk) bölündü. (10 tarif x 3 parça = 30)")

except Exception as e:
    print(f"❌ HATA: Veri yükleme veya parçalama hatası: {e}")
    sys.exit("Veri yükleme hatası.")

# === ADIM 5: EMBEDDING VE VEKTÖR DB (MODEL DEĞİŞİKLİĞİ) ===
print("\n--- ADIM 5: Embedding modeli yükleniyor ve Vektör DB oluşturuluyor... ---")
print("Lütfen bekleyin, 'MiniLM' modeli (yaklaşık 450 MB) indirilecek...")
start_time = time.time()

# 'mpnet' (başarısız) yerine 'MiniLM' (daha küçük) modelini kullanıyoruz
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
print(f"✅ Embedding modeli ({model_name}) yüklendi.")

vector_store = FAISS.from_documents(documents, embeddings)
print("✅ Vektör veritabanı (FAISS) 'MiniLM' ve 'Konteks Enjeksiyonu' ile başarıyla oluşturuldu!")
end_time = time.time()
print(f"(Bu işlem {end_time - start_time:.2f} saniye sürdü)")

# === ADIM 6: DEBUG KONTROLÜ (k=3) ===
print("\n--- ADIM 6: DEBUG - RETRIEVAL KONTROLÜ (k=3) ---")
print("Karnıyarık'a benzeyen EN İYİ 3 parçayı (chunk) arıyoruz:")
soru_test = "Karnıyarık tarifi"
try:
    benzer_belgeler = vector_store.similarity_search(soru_test, k=3)

    if benzer_belgeler:
        for i, doc in enumerate(benzer_belgeler):
            print(f"\n--- DÖNEN BELGE {i+1} (MUTLAKA 'Karnıyarık' içermeli) ---")
            print(doc.page_content[:150] + "...")
    else:
        print("Hiç belge dönemedi.")

except Exception as e:
    print(f"❌ DEBUG hatası: {e}")
print("\n--- DEBUG KONTROLÜ BİTTİ ---\n")

# === ADIM 7: RAG PIPELINE VE NİHAİ TEST (k=3) ===
print("--- ADIM 7: RAG Pipeline (k=3 ile) kuruluyor ve test ediliyor... ---")

llm = ChatGoogleGenerativeAI(model="models/gemini-flash-latest",
                             temperature=0.7,
                             top_p=0.85)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)
print("✅ RAG Pipeline (k=3 ile) başarıyla kuruldu.")

print("\n--- NİHAİ TEST (Konteks Enjeksiyonu + MiniLM) BAŞLIYOR ---")
soru_final = "Karnıyarık nasıl yapılır? Malzemeleri nelerdir?"
print(f"Soru: {soru_final}")

try:
    cevap = rag_pipeline.invoke(soru_final)
    print("\nCevap:")
    print(cevap['result'])
except Exception as e:
    print(f"❌ HATA: Model sorgulanamadı. Hata: {e}")

--- ADIM 1: Kütüphaneler kuruluyor... ---
✅ Kütüphaneler kuruldu.

--- ADIM 2: Kütüphaneler ve API Anahtarı yükleniyor... ---
✅ API Anahtarı ve Kütüphaneler Başarıyla Yüklendi!

--- ADIM 3: GitHub'dan güncel veri seti çekiliyor... ---
Cloning into 'akbank-genai-bootcamp-proje'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 11.68 KiB | 323.00 KiB/s, done.
Resolving deltas: 100% (2/2), done.
✅ Güncel repo klonlandı.

--- ADIM 4: Veri yükleniyor ve KONTEKST ENJEKSİYONU ile parçalanıyor... ---
10 adet ana tarif bulundu.
✅ Veri başarıyla 30 adet KONTEKSTLİ parçaya (chunk) bölündü. (10 tarif x 3 parça = 30)

--- ADIM 5: Embedding modeli yükleniyor ve Vektör DB oluşturuluyor... ---
Lütfen bekleyin, 'MiniLM' modeli (yaklaşık 450 MB) indirilecek...


/tmp/ipython-input-2425729563.py:73: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Embedding modeli (sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2) yüklendi.
✅ Vektör veritabanı (FAISS) 'MiniLM' ve 'Konteks Enjeksiyonu' ile başarıyla oluşturuldu!
(Bu işlem 17.72 saniye sürdü)

--- ADIM 6: DEBUG - RETRIEVAL KONTROLÜ (k=3) ---
Karnıyarık'a benzeyen EN İYİ 3 parçayı (chunk) arıyoruz:

--- DÖNEN BELGE 1 (MUTLAKA 'Karnıyarık' içermeli) ---
Başlık: Karnıyarık...

--- DÖNEN BELGE 2 (MUTLAKA 'Karnıyarık' içermeli) ---
Başlık: Menemen...

--- DÖNEN BELGE 3 (MUTLAKA 'Karnıyarık' içermeli) ---
Başlık: Hünkar Beğendi...

--- DEBUG KONTROLÜ BİTTİ ---

--- ADIM 7: RAG Pipeline (k=3 ile) kuruluyor ve test ediliyor... ---
✅ RAG Pipeline (k=3 ile) başarıyla kuruldu.

--- NİHAİ TEST (Konteks Enjeksiyonu + MiniLM) BAŞLIYOR ---
Soru: Karnıyarık nasıl yapılır? Malzemeleri nelerdir?

Cevap:
Sağlanan metinlerde Karnıyarık yemeğinin yapılışı veya malzemeleri hakkında bilgi bulunmamaktadır.


In [2]:
import sys
import time

# === ADIM 1: KURULUM ===
print("--- ADIM 1: Kütüphaneler kuruluyor... ---")
# Self-Querying için 'langchain-chroma' ve 'lark' ekliyoruz
!pip install -q -U google-generativeai langchain-google-genai langchain faiss-cpu langchain_community sentence-transformers langchain-chroma lark
print("✅ Kütüphaneler kuruldu.")

# === ADIM 2: API ANAHTARI VE IMPORTLAR ===
print("\n--- ADIM 2: Kütüphaneler ve API Anahtarı yükleniyor... ---")
import os
from google.colab import userdata
import google.generativeai as genai

# Gerekli LangChain importları
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
# YENİ: Vektör veritabanı olarak Chroma
from langchain_chroma import Chroma
# YENİ: SelfQueryRetriever
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

try:
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    print("✅ API Anahtarı ve Kütüphaneler Başarıyla Yüklendi!")
except Exception as e:
    print(f"❌ HATA: API Anahtarı yüklenemedi. 'Secrets' (🔑) bölümünü kontrol edin. Hata: {e}")
    sys.exit("API Anahtarı hatası.")

# === ADIM 3: GÜNCEL VERİYİ ÇEKME ===
print("\n--- ADIM 3: GitHub'dan güncel veri seti çekiliyor... ---")
!rm -rf akbank-genai-bootcamp-proje
!git clone https://github.com/mustafadevrim/akbank-genai-bootcamp-proje
print("✅ Güncel repo klonlandı.")

# === ADIM 4: KONTEKST ENJEKSİYONU PARÇALAMASI ===
print("\n--- ADIM 4: Veri yükleniyor ve KONTEKST ENJEKSİYONU ile parçalanıyor... ---")
try:
    with open("akbank-genai-bootcamp-proje/tarifler.txt", "r", encoding="utf-8") as f:
        tum_tarifler_metni = f.read()

    tarif_listesi = tum_tarifler_metni.split("\n---\n")
    print(f"{len(tarif_listesi)} adet ana tarif bulundu.")

    documents = [] # Parçaları (chunk) tutacak son liste

    for tarif_metni in tarif_listesi:
        if not tarif_metni.strip(): continue
        parts = tarif_metni.split("\nMalzemeler:\n", 1)
        if len(parts) < 2: continue
        baslik_content = parts[0].strip() # "Başlık: Karnıyarık"
        parts2 = parts[1].split("\nYapılışı:\n", 1)
        if len(parts2) < 2: continue
        malzemeler_content = parts2[0].strip()
        yapilis_content = parts2[1].strip()

        # Self-Querying için metadata'yı DÜZGÜN tanımlıyoruz
        # 'source' alanı "Başlık: Karnıyarık" metnini içerecek
        doc_metadata = {"source": baslik_content}

        # Stratejimiz (3 parça oluşturma) aynı
        documents.append(Document(page_content=baslik_content, metadata=doc_metadata))
        documents.append(Document(page_content=f"{baslik_content}\nMalzemeler:\n{malzemeler_content}", metadata=doc_metadata))
        documents.append(Document(page_content=f"{baslik_content}\nYapılışı:\n{yapilis_content}", metadata=doc_metadata))

    print(f"✅ Veri başarıyla {len(documents)} adet KONTEKSTLİ parçaya (chunk) bölündü.")

except Exception as e:
    print(f"❌ HATA: Veri yükleme veya parçalama hatası: {e}")
    sys.exit("Veri yükleme hatası.")

# === ADIM 5: EMBEDDING VE VEKTÖR DB (Chroma) ===
print("\n--- ADIM 5: Embedding modeli yükleniyor ve Vektör DB (Chroma) oluşturuluyor... ---")
print("Lütfen bekleyin, 'MiniLM' modeli (yaklaşık 450 MB) indirilecek...")
start_time = time.time()

# 'mpnet' (kafa karıştıran) yerine 'MiniLM' (hızlı ve yeterli) modelini kullanıyoruz
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
print(f"✅ Embedding modeli ({model_name}) yüklendi.")

# FAISS yerine Chroma veritabanını  kullanıyoruz
vector_store = Chroma.from_documents(documents, embeddings)
print("✅ Vektör veritabanı (Chroma) 'MiniLM' ve 'Konteks Enjeksiyonu' ile başarıyla oluşturuldu!")
end_time = time.time()
print(f"(Bu işlem {end_time - start_time:.2f} saniye sürdü)")

# === ADIM 6: SELF-QUERYING RETRIEVER KURULUMU ===
print("\n--- ADIM 6: Self-Querying Retriever kuruluyor... ---")

# LLM (Gemini)  modelini tanımla
llm = ChatGoogleGenerativeAI(model="models/gemini-flash-latest", temperature=0)

# Retriever'a metadata'mızın ne anlama geldiğini öğretiyoruz
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="Tarifin başlığı, örneğin 'Başlık: Karnıyarık' veya 'Başlık: Menemen'",
        type="string",
    ),
]
document_content_description = "Türk mutfağı yemek tarifleri"

# Retriever'ı oluştur
# Bu, LLM'i (Gemini) ve Vektör DB'yi (Chroma) birbirine bağlar
try:
    retriever = SelfQueryRetriever.from_llm(
        llm,
        vector_store,
        document_content_description,
        metadata_field_info,
        verbose=True # ÖNEMLİ: Gemini'nin filtreyi nasıl oluşturduğunu görmek için
    )
    print("✅ Self-Querying Retriever başarıyla kuruldu.")
except Exception as e:
    print(f"❌ HATA: Self-Querying Retriever kurulamadı: {e}")
    sys.exit("Retriever hatası.")

# === ADIM 7: RAG PIPELINE VE NİHAİ TEST ===
print("\n--- ADIM 7: RAG Pipeline kuruluyor ve test ediliyor... ---")

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, # Cevap üretecek model (Gemini)
    chain_type="stuff",
    retriever=retriever # Akıllı filtreleme yapan retriever
)
print("✅ RAG Pipeline (Self-Querying ile) başarıyla kuruldu.")

print("\n--- NİHAİ TEST (Self-Querying ile) BAŞLIYOR ---")
soru_final = "Karnıyarık nasıl yapılır? Malzemeleri nelerdir?"
print(f"Soru: {soru_final}")

try:
    cevap = rag_pipeline.invoke(soru_final)
    # Gemini'nin oluşturduğu filtreyi ve bulunan belgeleri göreceğiz (verbose=True sayesinde)
    print("\nCevap:")
    print(cevap['result'])
except Exception as e:
    print(f"❌ HATA: Model sorgulanamadı. Hata: {e}")

--- ADIM 1: Kütüphaneler kuruluyor... ---
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━